In [1]:
import operator
import sys
import time
import pp
from multiprocessing import Array, Manager

from math import exp

import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm as cm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import EllipticEnvelope
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline


In [2]:
_train_=pd.read_csv('../input/train.csv')
_train_.describe()

,row_id,x,y,accuracy,time,place_id
count,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07
mean,1.455901e+07,4.999770e+00,5.001814e+00,8.284912e+01,4.170104e+05,5.493787e+09
std,8.405649e+06,2.857601e+00,2.887505e+00,1.147518e+02,2.311761e+05,2.611088e+09
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000016e+09
25%,7.279505e+06,2.534700e+00,2.496700e+00,2.700000e+01,2.030570e+05,3.222911e+09
50%,1.455901e+07,5.009100e+00,4.988300e+00,6.200000e+01,4.339220e+05,5.518573e+09
75%,2.183852e+07,7.461400e+00,7.510300e+00,7.500000e+01,6.204910e+05,7.764307e+09
max,2.911802e+07,1.000000e+01,1.000000e+01,1.033000e+03,7.862390e+05,9.999932e+09


In [3]:
# testData=_train_[:100000]
# testData=testData.set_index('place_id').to_dict(orient='split')
# print [(p, len(testData[p])) for p in testData if len(testData[p]>1)]

In [4]:
_train_.sort(['time'], inplace=True)
train=_train_[:int(len(_train_)*0.9)]
test=_train_[int(len(_train_)*0.9):]

train=train[_train_['accuracy']>2]
places=train['place_id'].unique()
placeCounts=train['place_id'].value_counts()
train=train[(placeCounts[train['place_id']]>10).values]



train.loc[:,'day']=train.loc[:, 'time'].div(1440).map(int)
train.loc[:,'weekday']=train.loc[:,'day']%7
train.loc[:,'year']=train.loc[:,'day'].div(365).map(int)
train.loc[:,'month']=(train.loc[:,'day']%365).div(30).map(int)
train.loc[:,'hour']=train.loc[:,'time'].div(60).map(int)
train.loc[:, 'hourInDay']=train.loc[:,'hour']%24


test.loc[:, 'day']=test.loc[:,'time'].div(1440).map(int)
test.loc[:, 'weekday']=test.loc[:, 'day']%7
test.loc[:,'year']=test.loc[:,'day'].div(365).map(int)
test.loc[:,'month']=(test.loc[:,'day']%365).div(30).map(int)
test.loc[:, 'hour']=test.loc[:, 'time'].div(60).map(int)
test.loc[:, 'hourInDay']=test.loc[:,'hour']%24



/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/zhenjie/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/zhenjie/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [5]:
train.head(5)

,row_id,x,y,accuracy,time,place_id,day,weekday,year,month,hour,hourInDay
15280869,15280869,2.9374,1.0010,433,1,7186804346,0,0,0,0,0,0
23020268,23020268,5.3673,5.6998,25,1,7449653826,0,0,0,0,0,0
3977976,3977976,3.1148,3.6161,8,1,8090429732,0,0,0,0,0,0
27743788,27743788,3.7725,6.4504,65,2,4115006237,0,0,0,0,0,0
3836334,3836334,3.5102,3.6296,8,3,8090429732,0,0,0,0,0,0


In [6]:
g=train.groupby('place_id')
summary=pd.DataFrame()
summary['count']=g['y'].count()

summary['yMin']=g['y'].min()
summary['yMax']=g['y'].max()
summary['yMean']=g['y'].mean()
summary['yStd']=g['y'].std()

summary['xMin']=g['x'].min()
summary['xMax']=g['x'].max()
summary['xMean']=g['x'].mean()
summary['xStd']=g['x'].std()

summary['dayMin']=g['day'].min()
summary['dayMax']=g['day'].max()
summary['dayMean']=g['day'].mean()
summary['dayStd']=g['day'].std()

summary['weekdayMean']=g['weekday'].mean()
summary['weekdayStd']=g['weekday'].std()

summary['monthMean']=g['month'].mean()
summary['monthStd']=g['month'].std()

summary.sort_values(by='yMin', inplace=True, ascending=False)
summary.head(5)

,count,yMin,yMax,yMean,yStd,xMin,xMax,xMean,xStd,dayMin,dayMax,dayMean,dayStd,weekdayMean,weekdayStd,monthMean,monthStd
place_id,,,,,,,,,,,,,,,,,
2477740908,46,9.9798,9.9999,9.993546,0.004361,6.1437,8.2402,6.691791,0.386038,38,344,142.521739,60.870624,2.521739,2.062783,4.304348,2.031634
7761482869,37,9.9745,9.9998,9.990730,0.006718,3.2890,3.3576,3.327376,0.014952,0,495,222.324324,154.704926,3.243243,2.087069,4.027027,3.201445
9258213522,43,9.9739,10.0000,9.989940,0.007391,7.2536,7.8883,7.343521,0.088275,197,486,397.674419,70.482933,3.069767,1.778112,3.465116,3.686196
9548289088,47,9.9733,9.9998,9.989281,0.007792,2.9601,9.2086,3.960717,1.197568,237,500,446.085106,76.959004,2.851064,2.293457,3.851064,2.321719
5981474631,125,9.9732,9.9999,9.990237,0.006934,2.2136,5.2542,3.160953,0.304759,102,501,239.624000,128.814800,4.256000,2.047437,5.176000,1.601733


In [7]:
len(summary)

105498

In [8]:
fMaps=summary.to_dict()

countMap=fMaps['count']

yMaxMap=fMaps['yMax']
yMinMap=fMaps['yMin']
yMeanMap=fMaps['yMean']
yStdMap=fMaps['yStd']

xMaxMap=fMaps['xMax']
xMinMap=fMaps['xMin']
xMeanMap=fMaps['xMean']
xStdMap=fMaps['xStd']

dayMaxMap=fMaps['dayMax']
dayMinMap=fMaps['dayMin']
dayMeanMap=fMaps['dayMean']
dayStdMap=fMaps['dayStd']

weekdayMeanMap=fMaps['weekdayMean']
weekdayStdMap=fMaps['weekdayStd']

monthMeanMap=fMaps['monthMean']
monthStdMap=fMaps['monthStd']

In [9]:
nClasses=100
place_ids=set(summary.index[:nClasses])
place_list=summary.index[:nClasses]

subTrain=train[train['place_id'].isin(place_ids)]
subTest=test[test['place_id'].isin(place_ids)]


f=['x','y','accuracy', 'day', 'weekday', 'hourInDay']


In [11]:
summary.loc[1660219404,'yMax']


9.9846000000000004

In [12]:
f=['x','y','accuracy', 'day', 'weekday', 'hourInDay']
def regionalForestPred(pred, classes, regionalPoints, data):

    
    prob=[sorted(zip(p, classes))[-3:][::-1] for p in pred]
    
    
    regionalPred=[]
    for i in xrange(len(data)):
        sampleY=data['y'].iloc[i]
        sampleX=data['x'].iloc[i]
        sampleDay=data['day'].iloc[i]

        filteredPlaces=filter(lambda x: \
                              sampleY<=yMaxMap[x[1]]+yAllowance and sampleY>=yMinMap[x[1]]-yAllowance and \
                              sampleX<=xMaxMap[x[1]] and sampleX>=xMinMap[x[1]] and \
                              sampleDay<=dayMaxMap[x[1]] and sampleDay>=dayMinMap[x[1]], prob[i])
        if filteredPlaces:
            filteredPlaces=map(lambda x: [x[0]*regionalPoints, x[1]], filteredPlaces)
        regionalPred.append(filteredPlaces)
    return regionalPred

In [44]:
nClassesPerRegion=800
regionalClfs=[]
regionalYMax=[]
regionalYMin=[]
regionalPoints=[]
yAllowance=0


probas=[]

print 'total test data: {}'.format(len(subTest))
print 'total regions: {}'.format(nClasses/nClassesPerRegion)
globalStartTime=time.time()

nTotalPrediction=0

for i in (range(nClasses/nClassesPerRegion+1 if nClasses%nClassesPerRegion else nClasses/nClassesPerRegion)):
    print 'start regiona {}. Time: {}'.format(i+1, 0)
    currentTime=startingTime=time.time()
    
#   all place_ids in this regional training set
    regionalPlaces=place_list[i*nClassesPerRegion:(i+1)*nClassesPerRegion]
#     regional training samples and the max and min of Y values
    regionalTrain=subTrain[subTrain['place_id'].isin(regionalPlaces)]
    regionalYMax.append(regionalTrain['y'].max())
    regionalYMin.append(regionalTrain['y'].min())
    
#     test points that fall within the Ymin to Ymax region
    regionalTest=subTest[(subTest['y']<=regionalYMax[-1]) & (subTest['y']>=regionalYMin[-1])]
    nTotalPrediction+=len(regionalTest)
    print 'test Data in region {}: {}.'.format(i+1, len(regionalTest))
    currentTime=time.time()
    
# save the row number in the test set in order to index into the prediction array with the subTest index number.
    regionalTest.loc[:,'regionalRow']=xrange(len(regionalTest))
    
    print 'training on region {}.  Time elapsed: {}'.format(i+1, time.time()-currentTime)
    currentTime=time.time()
    
    
    
#     fit and get classifier
#     regionalClf=RandomForestClassifier(n_jobs=8, n_estimators=50, max_features=None).fit(regionalTrain[f], regionalTrain['place_id'])
    regionalClf=XGBClassifier(n_estimators=50, objective='multi:softprob').fit(regionalTrain[f], regionalTrain['place_id'])

# number of training samples in this region.  Used to compare assesss the likehood comparison between differnt regional predictions
    regionalPoints=len(regionalTrain)
    print 'predicting probability in region {}.  Time used in trainning: {}'.format(i+1, time.time()-currentTime)
    currentTime=time.time()

# predicting class probability for test sampels
    regionalPredict=regionalClf.predict_proba(regionalTest[f])
    
#     regionalPredict=zip(*regionalPredict)

    
    
    print 'prediction done. Filtering predictoins.  Time used in prediction: {}'.format(time.time()-currentTime)
    currentTime=time.time()
# filter and rankthe predictions for test samples.  and computer the performance in the region.
    regionalPredict=regionalForestPred(regionalPredict, regionalClf.classes_, regionalPoints, regionalTest)
    regionalAccurcy=1.0*sum([regionalTest['place_id'].iloc[i] in [p[1] for p in regionalPredict[i]] for i in xrange(len(regionalTest))])/len(regionalTest)
    regionalRelevance=1.0*sum([regionalTest['place_id'].iloc[i] in regionalPlaces for i in xrange(len(regionalTest))])/len(regionalTest)
    print 'regional top 3 prediction accuracy: ', regionalAccurcy
    print 'regional test set in-regoin ration: ', regionalRelevance
    print 'relevant accuracy: ', regionalAccurcy/regionalRelevance
    
# save probability predictions for all samples in subTest.  When sample is not the region, set proba pred to empty.
    print 'sorting predictions into global index'
    regionalRowIndex=set(regionalTest.index)  
    globalPredict=[regionalPredict[int(regionalTest.loc[int(t),'regionalRow'])] if t in regionalRowIndex else [] for t in subTest['row_id']]    
    probas.append(globalPredict)
    print 'region', i+1, 'fininshed.  Total time in regional {}: {}\n'.format(i+1, time.time()-startingTime)
    
    

print 'total predictions made: {}, prediction/test fold: {}'.format(nTotalPrediction, 1.0*nTotalPrediction/len(subTest))
print 'total time spent: {}'.format(time.time()-globalStartTime)

total test data: 35650
total regions: 2
start regiona 1. Time: 0
test Data in region 1: 29611.
training on region 1.  Time elapsed: 0.166171073914
predicting probability in region 1.  Time used in trainning: 606.689514875
prediction done. Filtering predictoins.  Time used in prediction: 73.0591938496
regional top 3 prediction accuracy:  0.419945290601
regional test set in-regoin ration:  0.526898787613
relevant accuracy:  0.797013203435
sorting predictions into global index
region 29611 fininshed.  Total time in regional 29611: 693.006731033

start regiona 2. Time: 0
test Data in region 2: 35618.
training on region 2.  Time elapsed: 0.182766914368
predicting probability in region 2.  Time used in trainning: 750.258466005
prediction done. Filtering predictoins.  Time used in prediction: 86.8926529884
regional top 3 prediction accuracy:  0.436745465776
regional test set in-regoin ration:  0.561626144084
relevant accuracy:  0.777644471106
sorting predictions into global index
region 35618

In [17]:
len(probas[0])

77140

In [45]:
# produce a list of (probability, place), each ranking top 3 of it's belonging region


probasAll=[reduce(operator.add, [probas[i][m] for i in xrange(len(probas))]) for m in xrange(len(subTest))]
# print probas[0]
# the same list stripped off probabilities
probasAll=[sorted(p)[::-1] for p in probasAll]
topPlaces=[[p[i][1] for i in xrange(len(p))] for p in probasAll]
print [len(probasAll[i]) for i in xrange(6)]

[5, 5, 4, 6, 6, 6]


In [46]:
# use probability in different regions to predict ranking
evaluation=[subTest['place_id'].iloc[i] in topPlaces[i] for i in xrange(len(subTest))]
1.0*sum(evaluation)/len(evaluation)
score=0
# have to use in topPlaces, because some itmes in topPlaces are empty.
1.0*sum([subTest['place_id'].iloc[i] in topPlaces[i][:1] for i in xrange(len(subTest))])/len(evaluation)

1.0*sum([subTest['place_id'].iloc[i] in topPlaces[i][:3] for i in xrange(len(subTest))])/len(evaluation)

score+=sum([subTest['place_id'].iloc[i] in topPlaces[i][:1] for i in xrange(len(subTest))])
score+=0.67*sum([subTest['place_id'].iloc[i] in topPlaces[i][1:2] for i in xrange(len(subTest))])
score+=0.33*sum([subTest['place_id'].iloc[i] in topPlaces[i][2:3] for i in xrange(len(subTest))])

1.0*score/len(evaluation)

# gnb=GaussianNB()
# print "here is the gnb"
# gnb.fit(subTrain[f], subTrain['place_id'])
# prob=gnb.predict_proba(subTest[f])

0.7851612903225806

0.4724544179523142

0.7212061711079943

0.6174305750350632

In [48]:
# use naive bayes to determine the ranking
def probability(d, mean, std, cycle=0):
    diff=min(abs(d-mean), cycle-abs(d-mean)) if cycle else d-mean
    return (2.713**(-diff**2/2/std**2))/(6.28**0.5)/std

subTestY=subTest['y'].tolist()
subTestX=subTest['x'].tolist()
subTestDay=subTest['day'].tolist()
subTestWeekday=subTest['weekday'].tolist()
subTestMonth=subTest['month'].tolist()




ranked=[]
for i in xrange(len(subTest)):
    tp=topPlaces[i]
    dist=sorted([[summary.loc[p,'count']*dayMaxMap[p]*probability(subTestY[i], yMeanMap[p], yStdMap[p])\
                  *probability(subTestX[i], xMeanMap[p], xStdMap[p])\
                  *probability(subTestDay[i], dayMeanMap[p], dayStdMap[p])\
                  *probability(subTestWeekday[i], weekdayMeanMap[p], weekdayStdMap[p], cycle=7)\
                  *probability(subTestMonth[i], monthMeanMap[p], monthStdMap[p], cycle=30),p] for p in tp])[::-1]
    
    ranked.append([p[1] for p in dist])
    

In [49]:

1.0*sum([subTest['place_id'].iloc[i] in ranked[i][:3] for i in xrange(len(subTest))])/len(evaluation)
score=0
score+=sum([subTest['place_id'].iloc[i] in ranked[i][:1] for i in xrange(len(subTest))])
score+=0.67*sum([subTest['place_id'].iloc[i] in ranked[i][1:2] for i in xrange(len(subTest))])
score+=0.33*sum([subTest['place_id'].iloc[i] in ranked[i][2:3] for i in xrange(len(subTest))])

1.0*score/len(evaluation)

0.7047685834502104

0.5694471248246844

In [31]:
# use knn for each point prediction

kNN=[]
for i in xrange(len(subTest)):
    tp=topPlaces[i]
    prob=sorted([[nbrClasses[knnPred[i].index(p)][0] if p in knnPred[i] else 0,p] for p in tp])[::-1]
    kNN.append([p[1] for p in prob])
print kNN[:3]

1.0*sum([subTest['place_id'].iloc[i] in kNN[i] for i in xrange(len(subTest))])/len(evaluation)
1.0*sum([subTest['place_id'].iloc[i] in kNN[i][:3] for i in xrange(len(subTest))])/len(evaluation)
score=0
score+=sum([subTest['place_id'].iloc[i] in kNN[i][:1] for i in xrange(len(subTest))])
score+=0.67*sum([subTest['place_id'].iloc[i] in kNN[i][1:2] for i in xrange(len(subTest))])
score+=0.33*sum([subTest['place_id'].iloc[i] in kNN[i][2:3] for i in xrange(len(subTest))])

1.0*score/len(evaluation)

NameError: name 'knnPred' is not defined

In [50]:
rank=[]
for i in xrange(len(subTest)):
    tmp=[]
    for np in xrange(len(topPlaces[i])):
        tmp.append([2*np+ranked[i].index(topPlaces[i][np]),topPlaces[i][np]])
    tmp.sort()
    rank.append([t[1] for t in tmp])
    
1.0*sum([subTest['place_id'].iloc[i] in rank[i][:1] for i in xrange(len(subTest))])/len(evaluation) 

1.0*sum([subTest['place_id'].iloc[i] in rank[i][:3] for i in xrange(len(subTest))])/len(evaluation) 
score=0
score+=sum([subTest['place_id'].iloc[i] in rank[i][:1] for i in xrange(len(subTest))])
score+=0.67*sum([subTest['place_id'].iloc[i] in rank[i][1:2] for i in xrange(len(subTest))])
score+=0.33*sum([subTest['place_id'].iloc[i] in rank[i][2:3] for i in xrange(len(subTest))])

1.0*score/len(evaluation)

0.5027769985974755

0.7269284712482469

0.6295447405329594

In [ ]:
# rf=RandomForestClassifier(n_jobs=8)
# rf.fit(subTrain[f], subTrain['place_id'])
# pred=rf.predict(subTest[f])
# subTest.loc[:,'pred']=pred
# accuracy=1.0*(subTest.loc[:,'place_id']==subTest.loc[:,'pred']).sum()/len(subTest)
# print accuracy

In [ ]:
# gbc=GradientBoostingClassifier()
# gbc.fit(subTrain[f], subTrain['place_id'])
# pred=gbc.predict(subTest[f])
# subTest.loc[:,'pred']=pred
# accuracy=1.0*(subTest['place_id']==subTest['pred']).sum()/len(subTest)
# print accuracy

In [12]:
nClasses=100
place_ids=set(summary.index[:nClasses])
place_list=summary.index[:nClasses]

subTrain=train[train['place_id'].isin(place_ids)]
subTest=test[test['place_id'].isin(place_ids)]


f=['x','y','accuracy', 'day', 'weekday', 'hourInDay']
s_w=(30-subTrain['day'].values/30.0)

xgbclf=xgb.XGBClassifier(objective='multi:softprob', weight=s_w)
xgbclf.fit(subTrain[f], subTrain['place_id'])
pred=xgbclf.predict(subTest[f])
subTest.loc[:,'pred']=pred
accuracy=1.0*(subTest['place_id']==subTest['pred']).sum()/len(subTest)
print accuracy

probas=xgbclf.predict


TypeError: __init__() got an unexpected keyword argument 'weight'

In [14]:
dir(xgb.XGBClassifier.__init__)

['__call__',
 '__class__',
 '__cmp__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__func__',
 '__get__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'im_class',
 'im_func',
 'im_self']

In [15]:
probas=xgbclf.predict_proba(subTest[f])

In [ ]:
# gnb=GaussianNB()
# gnb.fit(_train_[['x','y','time']], _train_['place_id'])